## Vicuna

### 1. Requirements

In [ ]:
!pip3 install fschat

### 2. LLaMA weight 다운로드

30GB RAM에서 작동할 수 있도록 Vicuna-7B를 이용한다. Vicuna-7B의 경우 llama-7b를 이용하여 vicuna weight를 추가하는 방법으로 얻을 수 있다.

LLaMA weights:
https://huggingface.co/decapoda-research


In [ ]:
## LLaMA weight 다운로드
git lfs install
git clone https://huggingface.co/decapoda-research/llama-7b-hf

#### 2.1. tokenizer_config.json 수정

`"tokenizer_class": "LLaMATokenizer"` -> `"tokenizer_class": "LlamaTokenizer"`

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./llama-7b-hf")

model = AutoModelForCausalLM.from_pretrained("./llama-7b-hf")

/home/zai-user/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 33/33 [00:21<00:00,  1.52it/s]


#### 2.2. Vicuna Weight Update

In [ ]:
python -m fastchat.model.apply_delta \
    --base-model-path ./llama-7b-hf \
    --target-model-path ./vicuna-7b \
    --delta-path lmsys/vicuna-7b-delta-v1.1

#### 2.3. Get Vicuna Model

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./vicuna-7b")

model = AutoModelForCausalLM.from_pretrained("./vicuna-7b")

/home/zai-user/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.24s/it]


### 2.4. Get Korean Vicuna Model

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("junelee/ko_vicuna_7b")
model = AutoModelForCausalLM.from_pretrained("junelee/ko_vicuna_7b")

/home/zai-user/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:57<00:00, 19.33s/it]


### 2.5 Make LLaVA Model

In [ ]:
git clone https://github.com/haotian-liu/LLaVA.git
cd LLaVA

In [ ]:
python3 -m llava.model.apply_delta \
    --base ../llama-7b-hf \
    --target ../llava-7b \
    --delta liuhaotian/LLaVA-7b-delta-v0

### 2.6 Get KoLLaVA (KoVicuna + CLIP) Model

In [1]:
"""
Usage:
python3 -m fastchat.model.apply_delta --base ~/model_weights/llama-7b --target ~/model_weights/vicuna-7b --delta lmsys/vicuna-7b-delta
"""
import argparse

import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from LLaVA.llava.model.llava import LlavaLlamaForCausalLM


def apply_delta(base_model_path, tokenizer_path, target_model_path, delta_path):
    print("Loading base model")
    base = AutoModelForCausalLM.from_pretrained(
        base_model_path, torch_dtype=torch.float16, low_cpu_mem_usage=True)

    print("Loading delta")
    delta = LlavaLlamaForCausalLM.from_pretrained(delta_path, torch_dtype=torch.float16, low_cpu_mem_usage=True)
    delta_tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

    print("Applying delta")
    for name, param in tqdm(delta.state_dict().items(), desc="Applying delta"):
        if name not in base.state_dict():
            assert name in ['model.mm_projector.weight', 'model.mm_projector.bias'], f'{name} not in base model'
            continue
        if param.data.shape == base.state_dict()[name].shape:
            param.data += base.state_dict()[name]
        else:
            assert name in ['model.embed_tokens.weight', 'lm_head.weight'], \
                f'{name} dimension mismatch: {param.data.shape} vs {base.state_dict()[name].shape}'
            bparam = base.state_dict()[name]
            param.data[:bparam.shape[0], :bparam.shape[1]] += bparam

    print("Saving target model")
    delta.save_pretrained(target_model_path)
    delta_tokenizer.save_pretrained(target_model_path)


In [2]:
apply_delta(
    base_model_path="junelee/ko_vicuna_7b",
    tokenizer_path="junelee/ko_vicuna_7b",
    target_model_path="./kollava-7b-v0",
    delta_path="liuhaotian/LLaVA-7b-delta-v0"
)

Loading base model


Loading checkpoint shards: 100%|██████████| 3/3 [00:59<00:00, 19.96s/it]
You are using a model of type llama to instantiate a model of type llava. This is not supported for all configurations of models and can yield errors.


Loading delta


Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.6.self_attn.q_proj.bias', 'text_model.encoder.layers.3.self_attn.q_proj.bias', 'text_model.encoder.layers.5.layer_norm1.bias', 'text_model.embeddings.position_embedding.weight', 'text_model.encoder.layers.4.layer_norm2.weight', 'text_model.encoder.layers.2.mlp.fc2.weight', 'text_model.encoder.layers.10.self_attn.v_proj.bias', 'text_model.encoder.layers.5.self_attn.k_proj.weight', 'text_model.encoder.layers.6.mlp.fc2.weight', 'text_model.encoder.layers.4.self_attn.k_proj.weight', 'text_model.encoder.layers.5.layer_norm2.bias', 'text_model.encoder.layers.0.self_attn.out_proj.bias', 'text_model.encoder.layers.6.self_attn.out_proj.bias', 'text_model.encoder.layers.1.self_attn.out_proj.bias', 'text_model.encoder.layers.4.mlp.fc1.bias', 'text_model.encoder.layers.7.self_attn.k_proj.weight', 'text_model.encoder.layers.4.mlp.fc2.bias', 'text_model.e

Applying delta


Applying delta: 100%|██████████| 325/325 [00:07<00:00, 46.01it/s]


Saving target model


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./kollava-7b-v0")
model = AutoModelForCausalLM.from_pretrained("./kollava-7b-v0")

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.6.self_attn.q_proj.bias', 'text_model.encoder.layers.3.self_attn.q_proj.bias', 'text_model.encoder.layers.5.layer_norm1.bias', 'text_model.embeddings.position_embedding.weight', 'text_model.encoder.layers.4.layer_norm2.weight', 'text_model.encoder.layers.2.mlp.fc2.weight', 'text_model.encoder.layers.10.self_attn.v_proj.bias', 'text_model.encoder.layers.5.self_attn.k_proj.weight', 'text_model.encoder.layers.6.mlp.fc2.weight', 'text_model.encoder.layers.4.self_attn.k_proj.weight', 'text_model.encoder.layers.5.layer_norm2.bias', 'text_model.encoder.layers.0.self_attn.out_proj.bias', 'text_model.encoder.layers.6.self_attn.out_proj.bias', 'text_model.encoder.layers.1.self_attn.out_proj.bias', 'text_model.encoder.layers.4.mlp.fc1.bias', 'text_model.encoder.layers.7.self_attn.k_proj.weight', 'text_model.encoder.layers.4.mlp.fc2.bias', 'text_model.e